In [1]:
using Revise
using Laplacians
using Pkg
@show Pkg.pathof(Laplacians)
using SparseArrays
using LinearAlgebra

┌ Info: Recompiling stale cache file /Users/rjkyng/.julia/compiled/v1.1/Laplacians/poVbr.ji for Laplacians [6f8e5838-0efe-5de0-80a3-5fb4f8dbb1de]
└ @ Base loading.jl:1184


Pkg.pathof(Laplacians) = "/Users/rjkyng/Dropbox/Repos/Laplacians.jl/src/Laplacians.jl"


In [3]:
function indexToLinear(x,y,z,xlen,ylen)
    return x+(y-1)*xlen+(z-1)*xlen*ylen
end

indexToLinear (generic function with 1 method)

In [7]:
s = 102

s1 = s
s2 = s
s3 = s
b1,b2,b3 = 3,3,3
w = 1e6
@time A = ggrid3_checkered(s1,s2,s3,b1,b2,b3,w);
x,y,z = ggrid3coords(s1,s2,s3);

n = s1*s2*s3
zsidesize = s1*s2
xsidesize = s2*s3
ysidesize = s1*s3
zside1b = 1:zsidesize # bot 
zside2b = n-zsidesize+1:n # top

zside1 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside1[i] = indexToLinear(x,y,1,s1,s2)
end

zside2 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside2[i] = indexToLinear(x,y,s3,s1,s2)
end

xside1 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
#     @show (i,(y,z))
#     if (y == 2 && z == 2)
#         @show z == 2
#         @show indexToLinear(1,y,z,s1,s2)
#     end
    xside1[i] = indexToLinear(1,y,z,s1,s2)
end

xside2 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
    xside2[i] = indexToLinear(s1,y,z,s1,s2)
end

yside1 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside1[i] = indexToLinear(x,1,z,s1,s2)
end

yside2 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside2[i] = indexToLinear(x,s2,z,s1,s2)
end

bndry = [zside1; zside2; xside1; xside2; yside1; yside2];
@time int = setdiff(1:n,bndry)

ni = length(int) #this is the remaining variables
@assert(ni == (s-2)^3)

@time L = lap(A) # create the Laplacian
@time M = L[int,int]
# remove the rows and colums corresponding to the boundary of the grid
# -- this is because we essentially want to force boundary voltages to be zero



  0.969339 seconds (48 allocations: 353.147 MiB, 55.36% gc time)
  0.258272 seconds (26 allocations: 26.643 MiB, 38.80% gc time)
  0.308463 seconds (22 allocations: 160.976 MiB, 31.45% gc time)
  0.325965 seconds (13 allocations: 121.622 MiB, 27.66% gc time)


1000000×1000000 SparseMatrixCSC{Float64,Int64} with 6940000 stored entries:
  [1      ,       1]  =  6.0e6
  [2      ,       1]  =  -1.0e6
  [101    ,       1]  =  -1.0e6
  [10001  ,       1]  =  -1.0e6
  [1      ,       2]  =  -1.0e6
  [2      ,       2]  =  6.0e6
  [3      ,       2]  =  -1.0e6
  [102    ,       2]  =  -1.0e6
  [10002  ,       2]  =  -1.0e6
  [2      ,       3]  =  -1.0e6
  [3      ,       3]  =  6.0e6
  [4      ,       3]  =  -1.0e6
  ⋮
  [999997 ,  999998]  =  -1.0e6
  [999998 ,  999998]  =  6.0e6
  [999999 ,  999998]  =  -1.0e6
  [989999 ,  999999]  =  -1.0e6
  [999899 ,  999999]  =  -1.0e6
  [999998 ,  999999]  =  -1.0e6
  [999999 ,  999999]  =  6.0e6
  [1000000,  999999]  =  -1.0e6
  [990000 , 1000000]  =  -1.0e6
  [999900 , 1000000]  =  -1.0e6
  [999999 , 1000000]  =  -1.0e6
  [1000000, 1000000]  =  6.0e6

In [9]:
b = randn(ni);

In [10]:
includet("../compare/hypreExport.jl")

In [11]:
matname = "hypreTestMatrix1"
vecname = "hypreTestVector1"
hypreExportMatrixVector(matname,M,vecname,b)


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018b (9.5.0.944444) 64-bit (maci64)
                              August 28, 2018

 
For online documentation, see https://www.mathworks.com/support
For product information, visit www.mathworks.com.
 
>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> Generating file: hypreTestMatrix1.00000
Generating file: hypreTestMatrix1.00001
>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> Generating file: hypreTestVector1.0
Generating INFO file: hypreTestVector1.INFO.0
Generating file: hypreTestVector1.1
Generating INFO file: hypreTestVector1.INFO.1
>> >> >> >> 

Process(`/Applications/MATLAB_R2018b.app/bin/matlab -nojvm '<' /Users/rjkyng/Dropbox/Repos/Laplacians.jl/src/../hypre/matlab2hypreMatrixVectorScript.m`, ProcessExited(0))